In [ ]:
!pip install icecream

In [ ]:
import sagemaker
import boto3
from icecream import ic
from PIL import Image
import numpy as np
import json

sm_client = boto3.client('sagemaker')
smr_client = boto3.client('sagemaker-runtime')
SOURCE_BUCKET_NAME = 'amazon-elasticache-police'
TARGET_BUCKET_NAME = sagemaker.session.Session().default_bucket()
PREFIX = 'amazon_elasticache_police/'
MODEL_TAR_NAME = 'amazon-elasticache-police-model.tar.gz'
MODEL_S3_URI = f's3://{TARGET_BUCKET_NAME}/{PREFIX}{MODEL_TAR_NAME}'
REGION_NAME = sagemaker.session.Session().boto_region_name
CONTAINER_URI = sagemaker.image_uris.retrieve(
    framework='tensorflow',
    region=REGION_NAME,version='2.4.1',
    image_scope='inference',
    instance_type='ml.c5.4xlarge'
)
SM_ROLE_ARN = sagemaker.get_execution_role()
ic(SOURCE_BUCKET_NAME)
ic(TARGET_BUCKET_NAME)
ic(PREFIX)
ic(MODEL_TAR_NAME)
ic(MODEL_S3_URI)
ic(REGION_NAME)
ic(CONTAINER_URI)
ic(SM_ROLE_ARN)

In [ ]:
# モデルを自分のアカウントのバケットにコピー
!aws s3 cp s3://{SOURCE_BUCKET_NAME}/{MODEL_TAR_NAME} {MODEL_S3_URI}

In [ ]:
MODEL_NAME = 'ElastiCachePoliceModel'
response = sm_client.create_model(
    ModelName=MODEL_NAME, # モデルの任意の名前
    PrimaryContainer={
        'Image': CONTAINER_URI, # コンテナの URI
        'ModelDataUrl': MODEL_S3_URI, # モデルの URI
    },
    ExecutionRoleArn=SM_ROLE_ARN, # ロールの ARN
)
print(response)

In [ ]:
ENDPOINT_CONFIG_NAME = 'ElastiCachePoliceEndpointConfig'
response = sm_client.create_endpoint_config(
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
    ProductionVariants=[
        {
            'VariantName': 'AllTrafic',
            'ModelName': MODEL_NAME,
            'InitialInstanceCount': 1,
            'InstanceType': 'ml.m5.large',
        },
    ],
)
print(response)

In [ ]:
ENDPOINT_NAME='ElastiCachePoliceEndpoint'
response = sm_client.create_endpoint(
    EndpointName=ENDPOINT_NAME,
    EndpointConfigName=ENDPOINT_CONFIG_NAME,
)
print(response)
waiter = sm_client.get_waiter('endpoint_in_service')
waiter.wait(EndpointName=ENDPOINT_NAME)

In [ ]:
img = Image.open('./test_data/AmazonElastiCache_ZenjidoJP-FeltPenLMT-TTF.ttf0.png')
no_problem_arr=((np.array(img)-127.5)/127.5).reshape(1,50,700,1)
img

In [ ]:
request_args = {
    'EndpointName': ENDPOINT_NAME,
    'ContentType' : 'application/json',
    'Accept' : 'application/json',
    'Body' : json.dumps({
        'instances':no_problem_arr.tolist()
    })
}
response = smr_client.invoke_endpoint(**request_args)
prediction_score=json.loads(response['Body'].read().decode('utf-8'))['predictions'][0][0]
ic(prediction_score)
print('alert') if prediction_score >= 0.5 else print('No Problem')

In [ ]:
img = Image.open('./test_data/AmazonElasticCache_ZenjidoJP-FeltPenLMT-TTF.ttf0.png')
alert_arr = ((np.array(img)-127.5)/127.5).reshape(1,50,700,1)
img

In [ ]:
request_args = {
    'EndpointName': ENDPOINT_NAME,
    'ContentType' : 'application/json',
    'Accept' : 'application/json',
    'Body' : str(alert_arr.tolist())
}
response = smr_client.invoke_endpoint(**request_args)
prediction_score=json.loads(response['Body'].read().decode('utf-8'))['predictions'][0][0]
ic(prediction_score)
print('alert') if prediction_score >= 0.5 else print('No Problem')

In [ ]:
MODEL_NAME = 'ElastiCachePoliceModel'
ENDPOINT_CONFIG_NAME = 'ElastiCachePoliceEndpointConfig'
ENDPOINT_NAME='ElastiCachePoliceEndpoint'
response = sm_client.delete_endpoint(EndpointName=ENDPOINT_NAME)
print(response)
response = sm_client.delete_endpoint_config(EndpointConfigName=ENDPOINT_CONFIG_NAME)
print(response)
response = sm_client.delete_model(ModelName=MODEL_NAME)
print(response)